In [ ]:
import numpy as np
import pandas as pd

from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
raw_data = pd.read_csv('/opt/ml/input/data/FE_total_data.csv')
raw_data.sort_values(by=["userID", "Timestamp"], inplace=True)
df = raw_data.copy()

- totalAnswer : csv 파일로 관리
- testMean : csv 파일로 관리
- testSum : csv 파일로 관리
- tagMean : csv 파일로 관리
- tagSum : csv 파일로 관리

In [ ]:
# 유저들의 문제 풀이수, 정답 수, 정답률을 시간순으로 누적해서 계산
df["user_correct_answer"] = df.groupby("userID")["answerCode"].transform(lambda x: x.cumsum().shift(1))
df["totalAnswer"] = df.groupby("userID")["answerCode"].cumcount()
df["user_acc"] = df["user_correct_answer"] / df["totalAnswer"]

    # testId와 KnowledgeTag의 전체 정답률은 한번에 계산
    # 아래 데이터는 제출용 데이터셋에 대해서도 재사용
correct_t = df.groupby(["testId"])["answerCode"].agg(["mean", "sum",'std'])
correct_t.columns = ["testMean", "testSum",'testStd']
correct_k = df.groupby(["KnowledgeTag"])["answerCode"].agg(["mean", "sum",'std'])
correct_k.columns = ["tagMean", "tagSum","tagStd"]
correct_a = df.groupby(["assessmentItemID"])["answerCode"].agg(["mean", "sum", "std"])
correct_a.columns = ['assessMean', 'assessSum', 'assessStd']


df = pd.merge(df, correct_t, on=["testId"], how="left")
df = pd.merge(df, correct_k, on=["KnowledgeTag"], how="left")
df = pd.merge(df, correct_a, on=["assessmentItemID"], how='left')
df

In [ ]:
df.to_csv('/opt/ml/input/data/FE/totalAnswer.csv', columns=["userID", "assessmentItemID", "testId", "answerCode", "Timestamp", "KnowledgeTag", "dataset", "totalAnswer"], index=False)
df.to_csv('/opt/ml/input/data/FE/testMean.csv', columns=["userID", "assessmentItemID", "testId", "answerCode", "Timestamp", "KnowledgeTag", "dataset", "testMean"], index=False)
df.to_csv('/opt/ml/input/data/FE/testSum.csv', columns=["userID", "assessmentItemID", "testId", "answerCode", "Timestamp", "KnowledgeTag", "dataset", "testSum"], index=False)
df.to_csv('/opt/ml/input/data/FE/testSum.csv', columns=["userID", "assessmentItemID", "testId", "answerCode", "Timestamp", "KnowledgeTag", "dataset", "testStd"], index=False)
df.to_csv('/opt/ml/input/data/FE/tagMean.csv', columns=["userID", "assessmentItemID", "testId", "answerCode", "Timestamp", "KnowledgeTag", "dataset", "tagMean"], index=False)
df.to_csv('/opt/ml/input/data/FE/tagSum.csv', columns=["userID", "assessmentItemID", "testId", "answerCode", "Timestamp", "KnowledgeTag", "dataset", "tagSum"], index=False)
df.to_csv('/opt/ml/input/data/FE/tagSum.csv', columns=["userID", "assessmentItemID", "testId", "answerCode", "Timestamp", "KnowledgeTag", "dataset", "tagStd"], index=False)
df.to_csv('/opt/ml/input/data/FE/assessMean.csv', columns=["userID", "assessmentItemID", "testId", "answerCode", "Timestamp", "KnowledgeTag", "dataset", "assessMean"], index=False)
df.to_csv('/opt/ml/input/data/FE/assessSum.csv', columns=["userID", "assessmentItemID", "testId", "answerCode", "Timestamp", "KnowledgeTag", "dataset", "assessSum"], index=False)
df.to_csv('/opt/ml/input/data/FE/assessStd.csv', columns=["userID", "assessmentItemID", "testId", "answerCode", "Timestamp", "KnowledgeTag", "dataset", "assessStd"], index=False)